In [1]:
from logging import getLogger
import os
import json
import pandas as pd
import time, datetime

from recbole.model.general_recommender.lightgcn import LightGCN

from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_logger, get_trainer, init_seed, set_color
from recbole.quick_start.quick_start import load_data_and_model
from recbole.quick_start import run_recbole
from recbole.config import Config
from recbole.data import create_dataset

from sklearn.metrics import accuracy_score, roc_auc_score

import torch

data = pd.read_csv('/home/siyun/ephemeral/code/recboleeee/amazon/amazon/amazon.inter', sep='\t')


/home/siyun/ephemeral/miniconda3/envs/recbole/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-15 00:44:16,224	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-15 00:44:16,356	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
userid, itemid = sorted(list(set(data['user_id:token']))), sorted(list(set(data['item_id:token'])))

n_user, n_item = len(userid), len(itemid)
userid_2_index = {v:i for i, v in enumerate(userid)}
itemid_2_index = {v:i+n_user for i, v in enumerate(itemid)}

id_2_index = dict(userid_2_index, **itemid_2_index)

index_2_id = {i: v for v, i in id_2_index.items()}
index_2_userid = {i: v for v, i in userid_2_index.items()}
index_2_itemid = {i: v for v, i in itemid_2_index.items()}

In [ ]:
itemid_2_index

In [ ]:
# import wandb
# from recbole.quick_start import run_recbole
# import yaml

# # Sweep을 위한 함수 정의
# def sweep():
#     sweep_config = {
#         'method': 'random',  # 랜덤 탐색 사용
#         'parameters': {
#             'epochs': {'values': [50, 100, 200]},  # 학습 에포크 수
#             'train_batch_size': {'values': [256, 512, 1024]},  # 학습 배치 크기
#             'learning_rate': {'values': [0.001, 0.01, 0.1]},  # 학습률 설정
#             'weight_decay': {'values': [0.001, 0.01, 0.1]},  # 가중치 감쇠 설정
#         }
#     }
    
#     # sweep_config를 sweep.yaml 파일로 저장
#     with open('sweep.yaml', 'w') as f:
#         yaml.dump(sweep_config, f)
    
#     # Sweep 실행
#     sweep_id = wandb.sweep(sweep_config, project='my-recbole-project')  # wandb 프로젝트 이름 설정
    
#     # Sweep 결과 출력

# # Sweep 함수 실행
# sweep()

In [4]:
# RecBole의 학습/평가 설정을 정의합니다.
config_dict = {
    'data_path': '/home/siyun/ephemeral/code/recboleeee/amazon',  # 데이터셋이 위치한 폴더 경로
    'dataset': 'amazon',   # 데이터셋 이름
    'log_wandb': True,  # WandB를 사용하여 로그 기록 여부
    'epochs': 50,  # 학습 에포크 수
    'train_batch_size': 256,  # 학습 배치 크기
    # 'train_batch_size': 1024,  # 학습 배치 크기
    'learner': 'adam',  # Optimizer 설정
    'learning_rate': 0.001,  # 학습률 설정
    'eval_step': 1,  # 검증 간격 설정
    'stopping_step': 10,  # 검증 기반 조기 종료 임계값 설정
    'weight_decay': 0.0001,  # 가중치 감쇠 설정
    'loss_decimal_place': 4,  # 손실 값의 소수 자릿수 설정
    'loss_type': 'CE',
    'train_neg_sample_args': {'neg_item_sample_num': 0},
    'user_id_field': 'user_id:token',  # 사용자 ID 필드 설정
    'item_id_field': 'item_id:token',  # 아이템 ID 필드 설정
    'rating_field': 'ratings:float',  # 평점 필드 설정
    'userid_2_index': userid_2_index,  # 사용자 ID를 인덱스로 매핑하는 사전 설정
    'itemid_2_index': itemid_2_index  # 아이템 ID를 인덱스로 매핑하는 사전 설정
}


# RecBole을 사용하여 모델을 학습하고 평가합니다.
run_recbole(model='LightGCN', dataset='amazon', config_dict=config_dict)


15 Mar 00:46    INFO  ['/home/siyun/ephemeral/miniconda3/envs/recbole/lib/python3.8/site-packages/ipykernel_launcher.py', '--f=/home/siyun/.local/share/jupyter/runtime/kernel-v2-1153841fF0SV00fnXdJ.json']
15 Mar 00:46    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /home/siyun/ephemeral/code/recboleeee/amazon/amazon
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 50
train_batch_size = 256
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'neg_item_sample_num': 0, 'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0001
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_b

In [16]:
import torch
import pandas as pd
from recbole.model.general_recommender import LightGCN
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

from recbole.utils import init_seed, init_logger
from recbole.data.interaction import Interaction

# 모델 설정과 데이터셋 준비
data_path = '/home/siyun/ephemeral/code/recbole/gender_inf'
dataset_name = 'gender_inf'  

config_dict = {
    'data_path': data_path,  
    'dataset': dataset_name,
    'embedding_size' : 64  
}
# config = Config(model='LightGCN', dataset='gender_inf', config_file_list=['your_config_file.yaml'])
config = Config(model='LightGCN', config_dict=config_dict)
init_seed(config['seed'], config['reproducibility'])
init_logger(config)


dataset = create_dataset(config)

# 모델 로드
model = LightGCN(config, dataset).to(config['device'])
checkpoint = torch.load('/home/siyun/ephemeral/code/saved/LightGCN-Mar-05-2024_23-57-57.pth')
model.load_state_dict(checkpoint['state_dict'])

# 전체에 대한 예측 수행
model.eval()  # 평가 모드로 설정

gift_df = pd.read_csv('/home/siyun/ephemeral/data/gift_group.csv')

gift_df.rename(columns={'product_id': 'item_id', 'user_id:token': 'user_id'}, inplace=True)


required_fields = ['user_id', 'item_id']
interaction_data = {field: torch.tensor(gift_df[field].values) for field in required_fields}

interaction = Interaction(interaction_data)

with torch.no_grad():
    predictions = model.predict(interaction)

gift_df['predicted_score'] = predictions.cpu().numpy()
gift_df.to_csv('/home/siyun/ephemeral/code/saved/updated_gift_group.csv', index=False)


# full_data = dataset.build_full()    # 전체 데이터셋에 대한 interaction 객체 생성
# full_interaction = full_data.interaction.to(config['device'])   

# with torch.no_grad():
#     full_predictions = model.predict(full_interaction)

# # 결과를 gift 데이터프레임에 추가하고 저장 (예시는 pandas로 진행)
# import pandas as pd

# gift_df = pd.read_csv('/path/to/gift_group.csv')
# gift_df['predicted_score'] = full_predictions.cpu().numpy()
# gift_df.to_csv('/path/to/updated_gift_group.csv', index=False)


../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [32,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [33,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [34,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [35,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [36,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
../aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [37,0,0] Assertion 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
import argparse
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    #feat siyun : train에서 진행했던 모델의 결과를 여기에 입력 
    parser.add_argument('--model_path', '-m', type=str, default='/data/ephemeral/RECBOLE/RecBole/saved/LightGCN-Feb-21-2024_02-19-02.pth', help='name of models')
    # /data/ephemeral/RECBOLE/RecBole/saved/EASE-Feb-17-2024_02-38-17_best.pth
    
    args, _ = parser.parse_known_args()
    
    # model, dataset 불러오기
    config, model, dataset, train_data, valid_data, test_data = load_data_and_model(args.model_path)
    
    # device 설정
    device = config.final_config_dict['device']
    
    # user, item id -> token 변환 array
    user_id2token = dataset.field2id_token['user_id']
    item_id2token = dataset.field2id_token['item_id']
    
    # user-item sparse matrix
    matrix = dataset.inter_matrix(form='csr')

    # user id, predict item id 저장 변수
    pred_list = None
    user_list = None
    
    model.eval()
    for data in test_data:
        interaction = data[0].to(device)
        
        score = model.full_sort_predict(interaction)
        
        rating_pred = score.cpu().data.numpy().copy()
        batch_user_index = interaction['user_id'].cpu().numpy()
        rating_pred[matrix[batch_user_index].toarray() > 0] = 0
        ind = np.argpartition(rating_pred, -10)[:, -10:]
        
        arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]

        arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]

        batch_pred_list = ind[
            np.arange(len(rating_pred))[:, None], arr_ind_argsort
        ]
        
        # 예측값 저장
        if pred_list is None:
            pred_list = batch_pred_list
            user_list = batch_user_index
        else:
            pred_list = np.append(pred_list, batch_pred_list, axis=0)
            user_list = np.append(user_list, batch_user_index, axis=0)
        
    result = []
    for user, pred in zip(user_list, pred_list):
        for item in pred:
            result.append((int(user_id2token[user]), int(item_id2token[item])))
            
    # 데이터 저장
    dataframe = pd.DataFrame(result, columns=["user", "item"])
    dataframe.sort_values(by='user', inplace=True)
    # 저장될 파일의 경로 지정
    ## ex) /data/ephemeral/RECBOLE/RecBole/saved/EASE_hyper_submission.csv
    dataframe.to_csv(
        "/data/ephemeral/RECBOLE/RecBole/saved/lightgcn_epoch93.csv", index=False
    )
    
    print('inference done!')